In [24]:
# Installing and defining libraries
library(tidyverse)
library(fixest)


# Loading data
data_agg <- read_rds('CS3.rds')

In [4]:
pro <- data_agg %>%
    mutate(fefe = ifelse(year > 2015, 1, 0))

In [20]:
pro2 <- pro %>%
    filter(!is.na(shareAA))

In [10]:
colnames(pro2)

[1] "origin"        "finaldest"     "return"        "year"         
 [5] "airports"      "return_sym"    "stops"         "ptotalAA"     
 [9] "ptotalUS"      "ptotallargest" "passengers"    "itinfare"     
[13] "after"         "before"        "avgprice"      "shareAA"      
[17] "shareUS"       "sharelargest"  "AA"            "US"           
[21] "AA_and_US"     "AA_or_US"      "market"        "pass_bef"     
[25] "pass_aft"      "balanced"      "treated"       "untreated"    
[29] "smallmkt"      "lnavgp"        "d_lnavgp"      "fefe"

In [16]:
reg <- lm(lnavgp ~ shareAA, data=pro2, weights = pass_bef)

In [23]:
data_agg <- data_agg %>%
  group_by(market) %>%
  mutate(price_missing_tag = ifelse(any(is.na(lnavgp)), 1, 0)) %>%
  ungroup()


In [6]:
data_balanced <- data_agg %>%
  filter(balanced == 1, price_missing_tag == 0)


In [7]:
nrow(filter(data_balanced, year == 2016, treated == 0, untreated == 0))
nrow(filter(data_balanced, year == 2016, treated == 1, untreated == 0))
nrow(filter(data_balanced, year == 2016, treated == 0, untreated == 1))
nrow(filter(data_balanced, year == 2016, treated == 1, untreated == 1))

[1] 29304

[1] 11628

[1] 71700

[1] 0

In [8]:
data_final <- data_balanced %>%
  filter((treated==0 & untreated==1) | (treated==1 & untreated==0))


In [10]:
small <- data_final %>% filter(smallmkt==1)
large <- data_final %>% filter(smallmkt==0)


In [22]:
# Running
reg_total <- feols(lnavgp ~ treated*after, weights = data_final$pass_bef, data = data_final, vcov="HC1")
reg_small <- feols(lnavgp ~ treated*after, weights = small$pass_bef, data = small, vcov="HC1")
reg_large <- feols(lnavgp ~ treated*after, weights = large$pass_bef, data = large, vcov="HC1")

ERROR: Error in feols(lnavgp ~ treated * after, weights = data_final$pass_bef, : Argument 'data' could not be evaluated.
PROBLEM: object 'data_final' not found.


In [13]:
# Presenting
etable(reg_total , reg_small, reg_large, 
       headers = c("All markets", "Small markets", "Large markets"), digits = 3 )

# Related DiD averages
data_final %>%
  group_by(after, treated) %>%
  summarise(sprintf("%.3f", weighted.mean(lnavgp, pass_bef, na.rm = TRUE)), n())


,,reg_total,reg_small,reg_large
,<chr>,<chr>,<chr>,<chr>
1,,All markets,Small markets,Large markets
2,Dependent Var.:,lnavgp,lnavgp,lnavgp
3,,,,
4,Constant,4.77*** (0.042),5.03*** (0.021),3.45*** (0.114)
5,treated,0.196*** (0.045),0.227*** (0.024),1.19*** (0.117)
6,after,0.153** (0.058),0.133*** (0.030),0.254. (0.151)
7,treated x after,-0.176** (0.062),-0.161*** (0.035),-0.272. (0.156)
8,_______________,________________,_________________,_______________
9,S.E. type,Heterosked.-rob.,Heteroskeda.-rob.,Heteroske.-rob.


`summarise()` has grouped output by 'after'. You can override using the
`.groups` argument.


after,treated,"sprintf(""%.3f"", weighted.mean(lnavgp, pass_bef, na.rm = TRUE))",n()
<dbl>,<dbl>,<chr>,<int>
0,0,4.766,71700
0,1,4.963,11628
1,0,4.919,71700
1,1,4.940,11628
